# Recoinize

[https://github.com/nicknochnack/FaceRecognition](https://github.com/nicknochnack/FaceRecognition/blob/main/Facial%20Verification%20with%20a%20Siamese%20Network%20-%20Final.ipynb)

# Setup

## Pull code from GitHub

In [ ]:
from src.helpers.cmd import git_update, print_gpu_name
from src.helpers.install import install_requirements
git_update("live-edit", force=False, show_output=False)
# install_requirements()

In [6]:
print_gpu_name()

'NVIDIA RTX A4000\n'

## Import dependencies

In [7]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

# Import tensorflow dependencies - Functional API
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from keras.models import Sequential, load_model, Model
from datasets import load_dataset

from src.models import make_embedding, make_siamese_model
from src.helpers.setup import set_gpus_growth
from src.dataset import preprocess, create_pairs

set_gpus_growth()

# Get and preprocess data

In [ ]:
# Load the dataset from HuggingFace
dataset = load_dataset('photonsquid/coins-euro')
dataset = dataset.with_format('tf')

In [10]:
# Split the dataset into train and test
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# Preprocess the data
train_dataset['image'].map(preprocess)
test_dataset['image'].map(preprocess)

In [ ]:
# create the pairs and labels for the training dataset
train_dataset.map(create_pairs)

# create the pairs and labels for the testing dataset
test_pairs, test_labels = create_pairs(test_dataset['image'], test_dataset['label'])

In [ ]:
# prepare the dataset for training
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# prepare the dataset for testing
test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)